In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import random
from collections import deque
import pandas as pd

class ProcessSchedulerGUI:
    def __init__(self, master):
        self.master = master
        self.master.title("Mô phỏng Thuật toán Lập lịch CPU")

        # Nhập số lượng tiến trình
        self.num_processes_label = ttk.Label(master, text="Số lượng tiến trình:")
        self.num_processes_label.grid(row=0, column=0, padx=5, pady=5, sticky="w")
        self.num_processes_entry = ttk.Entry(master)
        self.num_processes_entry.grid(row=0, column=1, padx=5, pady=5, sticky="ew")
        self.num_processes_entry.insert(0, "3")

        # Chọn cách nhập dữ liệu
        self.input_method_label = ttk.Label(master, text="Chọn phương thức nhập:")
        self.input_method_label.grid(row=1, column=0, padx=5, pady=5, sticky="w")
        self.input_method_combo = ttk.Combobox(master, values=["Nhập tay", "Sinh ngẫu nhiên"])
        self.input_method_combo.grid(row=1, column=1, padx=5, pady=5, sticky="ew")
        self.input_method_combo.set("Nhập tay")

        # Khung nhập thông tin tiến trình
        self.process_data_frame = ttk.LabelFrame(master, text="Thông tin tiến trình")
        self.process_data_frame.grid(row=2, column=0, columnspan=2, padx=5, pady=5, sticky="nsew")

        # Chọn thuật toán
        self.algorithm_label = ttk.Label(master, text="Chọn thuật toán:")
        self.algorithm_label.grid(row=3, column=0, padx=5, pady=5, sticky="w")
        self.algorithm_combo = ttk.Combobox(master, values=["FCFS", "RR"])
        self.algorithm_combo.grid(row=3, column=1, padx=5, pady=5, sticky="ew")
        self.algorithm_combo.set("FCFS")

        # Time quantum cho RR
        self.quantum_label = ttk.Label(master, text="Time Quantum (cho RR):")
        self.quantum_label.grid(row=4, column=0, padx=5, pady=5, sticky="w")
        self.quantum_entry = ttk.Entry(master)
        self.quantum_entry.grid(row=4, column=1, padx=5, pady=5, sticky="ew")
        self.quantum_entry.insert(0, "2")

        # Nút mô phỏng
        self.simulate_button = ttk.Button(master, text="Mô phỏng", command=self.simulate)
        self.simulate_button.grid(row=5, column=0, columnspan=2, padx=5, pady=10, sticky="ew")

        # Khung kết quả
        self.results_frame = ttk.LabelFrame(master, text="Kết quả mô phỏng")
        self.results_frame.grid(row=6, column=0, columnspan=2, padx=5, pady=5, sticky="nsew")

        self.results_tree = ttk.Treeview(self.results_frame, columns=("ID", "Arrival", "Burst", "Start", "Finish", "Waiting", "TAT"))
        for col in self.results_tree["columns"]:
            self.results_tree.heading(col, text=col)
            self.results_tree.column(col, width=80)
        self.results_tree.pack(fill="both", expand=True)

        self.avg_wait_label = ttk.Label(self.results_frame, text="")
        self.avg_wait_label.pack(padx=5, pady=2, anchor="w")
        self.avg_tat_label = ttk.Label(self.results_frame, text="")
        self.avg_tat_label.pack(padx=5, pady=2, anchor="w")

        # Gantt Chart
        self.gantt_label = ttk.Label(master, text="")
        self.gantt_label.grid(row=7, column=0, columnspan=2, padx=5, pady=5)

        # Các biến trung gian
        self.process_inputs = []
        self.update_process_inputs()

        self.num_processes_entry.bind("<Return>", self.update_process_inputs)
        self.input_method_combo.bind("<<ComboboxSelected>>", self.update_process_inputs)

        master.grid_columnconfigure(1, weight=1)
        self.process_data_frame.grid_columnconfigure(2, weight=1)
        self.results_frame.grid_columnconfigure(0, weight=1)

    def update_process_inputs(self, event=None):
        try:
            num = int(self.num_processes_entry.get())
        except ValueError:
            return

        for widget in self.process_data_frame.winfo_children():
            widget.destroy()
        self.process_inputs.clear()

        if self.input_method_combo.get() == "Nhập tay":
            for i in range(num):
                ttk.Label(self.process_data_frame, text=f"P{i+1} Arrival:").grid(row=i, column=0, padx=5, pady=2, sticky="w")
                arrival = ttk.Entry(self.process_data_frame)
                arrival.grid(row=i, column=1, padx=5, pady=2, sticky="ew")
                arrival.insert(0, "0")

                ttk.Label(self.process_data_frame, text="Burst:").grid(row=i, column=2, padx=5, pady=2, sticky="w")
                burst = ttk.Entry(self.process_data_frame)
                burst.grid(row=i, column=3, padx=5, pady=2, sticky="ew")
                burst.insert(0, "3")

                self.process_inputs.append({'arrival': arrival, 'burst': burst})
        else:
            processes = []
            for i in range(num):
                pid = f"P{i+1}"
                arrival = random.randint(0, 5)
                burst = random.randint(1, 10)
                processes.append({'id': pid, 'arrival': arrival, 'burst': burst})
            processes.sort(key=lambda x: x['arrival'])

            for i, p in enumerate(processes):
                ttk.Label(self.process_data_frame, text=f"{p['id']} Arrival:").grid(row=i, column=0, padx=5, pady=2, sticky="w")
                arrival = ttk.Entry(self.process_data_frame)
                arrival.grid(row=i, column=1, padx=5, pady=2, sticky="ew")
                arrival.insert(0, str(p['arrival']))

                ttk.Label(self.process_data_frame, text="Burst:").grid(row=i, column=2, padx=5, pady=2, sticky="w")
                burst = ttk.Entry(self.process_data_frame)
                burst.grid(row=i, column=3, padx=5, pady=2, sticky="ew")
                burst.insert(0, str(p['burst']))

                self.process_inputs.append({'arrival': arrival, 'burst': burst})

    def get_process_data(self):
        processes = []
        for i, inputs in enumerate(self.process_inputs):
            try:
                arrival = int(inputs['arrival'].get())
                burst = int(inputs['burst'].get())
                processes.append({'id': f'P{i+1}', 'arrival': arrival, 'burst': burst})
            except ValueError:
                messagebox.showerror("Lỗi", "Nhập số nguyên hợp lệ.")
                return None
        return sorted(processes, key=lambda x: x['arrival'])

    def simulate_fcfs(self, processes):
        processes.sort(key=lambda x: (x['arrival'], x['id']))
        current_time = 0
        gantt_chart = []
        results = []
        for p in processes:
            if current_time < p['arrival']:
                gantt_chart.append(('Idle', current_time, p['arrival']))
                current_time = p['arrival']
            start = current_time
            end = start + p['burst']
            gantt_chart.append((p['id'], start, end))
            results.append({
                'ID': p['id'], 'Arrival': p['arrival'], 'Burst': p['burst'],
                'Start': start, 'Finish': end,
                'Waiting': start - p['arrival'],
                'TAT': end - p['arrival']
            })
            current_time = end

        avg_tat = sum(r['TAT'] for r in results) / len(results)
        avg_wt = sum(r['Waiting'] for r in results) / len(results)

        return results, gantt_chart, avg_tat, avg_wt

    def simulate_rr(self, processes, quantum):
        processes.sort(key=lambda x: (x['arrival'], x['id']))
        ready_queue = deque()
        remaining = {p['id']: p['burst'] for p in processes}
        completion = {}
        gantt_chart = []
        start_times = {p['id']: None for p in processes}
        time = 0
        i = 0
        n = len(processes)
        while len(completion) < n:
            while i < n and processes[i]['arrival'] <= time:
                ready_queue.append(processes[i])
                i += 1
            if not ready_queue:
                if i < n:
                    next_arrival = processes[i]['arrival']
                    gantt_chart.append(('Idle', time, next_arrival))
                    time = next_arrival
                    continue
            proc = ready_queue.popleft()
            pid = proc['id']
            if start_times[pid] is None:
                start_times[pid] = time
            run_time = min(quantum, remaining[pid])
            gantt_chart.append((pid, time, time + run_time))
            time += run_time
            remaining[pid] -= run_time
            while i < n and processes[i]['arrival'] <= time:
                ready_queue.append(processes[i])
                i += 1
            if remaining[pid] == 0:
                completion[pid] = time
            else:
                ready_queue.append(proc)

        results = []
        for p in processes:
            comp = completion[p['id']]
            results.append({
                'ID': p['id'], 'Arrival': p['arrival'], 'Burst': p['burst'],
                'Start': start_times[p['id']], 'Finish': comp,
                'Waiting': comp - p['arrival'] - p['burst'],
                'TAT': comp - p['arrival']
            })

        avg_tat = sum(r['TAT'] for r in results) / len(results)
        avg_wt = sum(r['Waiting'] for r in results) / len(results)

        return results, gantt_chart, avg_tat, avg_wt

    def simulate(self):
        processes = self.get_process_data()
        if not processes:
            return

        algo = self.algorithm_combo.get()
        if algo == "FCFS":
            results, gantt_chart, avg_tat, avg_wt = self.simulate_fcfs(processes)
        elif algo == "RR":
            quantum = int(self.quantum_entry.get())
            results, gantt_chart, avg_tat, avg_wt = self.simulate_rr(processes, quantum)
        else:
            messagebox.showerror("Lỗi", "Thuật toán không hỗ trợ.")
            return

        for item in self.results_tree.get_children():
            self.results_tree.delete(item)
        for r in results:
            self.results_tree.insert("", "end", values=(r['ID'], r['Arrival'], r['Burst'], r['Start'], r['Finish'], r['Waiting'], r['TAT']))

        self.avg_wait_label.config(text=f"Waiting Time TB: {avg_wt:.2f}")
        self.avg_tat_label.config(text=f"Turnaround Time TB: {avg_tat:.2f}")

        gantt_str = "Gantt Chart: " + " ".join([f"[{p[0]}]({p[1]}→{p[2]})" for p in gantt_chart])
        self.gantt_label.config(text=gantt_str)

if __name__ == "__main__":
    root = tk.Tk()
    app = ProcessSchedulerGUI(root)
    root.mainloop()
